### Script to load monthly netcdf flight data and combine into single pandas dataframe

In [2]:
import xarray as xr
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import os
os.chdir('/p/project1/deepacf/stephenson1/ESDP-FinalProject/')
os.getcwd()

'/p/project1/deepacf/stephenson1/ESDP-FinalProject'

In [4]:
## loop through folders and files in folder and add to list
folders = sorted(glob("/p/scratch/deepacf/external_data/IAGOS/*/", recursive = True))

data_list = []
for i in range(0, len(folders)):
#for i in range(0, 2):
    file_list = sorted(glob(folders[i] + "/*.nc"))
    for j in range(0, len(file_list)):
        nc_file = xr.open_dataset(file_list[j])
        df = nc_file.to_dataframe()
        data_list.append(df)
        print(file_list[j])
    #print(folders[i])

/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996011512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996021512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996031512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996041512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996051512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996061512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996071512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996081512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996091512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996101512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996111512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1996/iagos_1996121512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1997/iagos_1997011512_ERA5.nc
/p/scratch/deepacf/external_data/IAGOS/HY1997/iagos_1997021512_ERA5.nc
/p/scr

In [5]:
## concatenate list into dataframe
data_all = pd.concat(data_list)

In [6]:
del data_list

In [7]:
## select columns of interest
data_all_select = data_all.loc[:, ['air_press', 'lon', 'lat', 'air_temp', 'air_temp_flag',
                                  'RHI', 'RHI_flag', 'RHL', 'RHL_flag', 'baro_alt']]

In [8]:
## remove missing values
data_all_select = data_all_select.dropna()

## filter to desired pressure level
data_all_select = data_all_select[(data_all_select['air_press'] >= 150) & 
                                    (data_all_select['air_press'] <= 400)]

## filter to north atlantic
data_all_select = data_all_select[(data_all_select['lat'] >= 0) & (data_all_select['lat'] <= 68) &
                                    (data_all_select['lon'] >= -80) & (data_all_select['lon'] <= 0)]

## filter temperature and humidity values
data_all_select = data_all_select[(data_all_select['RHL'] >= 0) & (data_all_select['RHL'] <= 1)]
data_all_select = data_all_select[(data_all_select['RHI'] >= 0) & (data_all_select['RHI'] <= 1)]
data_all_select = data_all_select[(data_all_select['air_temp'] >= 193) & 
                                    (data_all_select['air_temp'] <= 333)]

In [9]:
data_all_select.head()

,air_press,lon,lat,air_temp,air_temp_flag,RHI,RHI_flag,RHL,RHL_flag,baro_alt
time,,,,,,,,,,
1996-01-01 13:42:04,237.899994,-2.02,55.00,215.410004,13.0,0.995217,10.0,0.548,12.0,10682.000000
1996-01-01 13:42:08,237.800003,-2.02,55.00,215.460007,13.0,0.966160,10.0,0.532,12.0,10682.900391
1996-01-01 13:42:12,237.800003,-2.04,55.02,215.369995,13.0,0.939358,10.0,0.516,12.0,10684.200195
1996-01-01 13:42:16,237.800003,-2.04,55.02,215.119995,13.0,0.915692,10.0,0.503,12.0,10684.200195
1996-01-01 13:42:20,237.800003,-2.06,55.02,214.880005,13.0,0.892026,10.0,0.490,12.0,10684.200195


In [10]:
len(data_all.index) - len(data_all_select.index)

324315891

In [11]:
## save dataframe as pickle for analysis
data_all_select.to_pickle("/p/project1/deepacf/stephenson1/ESDP-FinalProject/iagos_combined.pkl")